In [ ]:
%pip install git-lfs --upgrade
%pip install gitpython

# Useful packages to be installed

In order to clone the repo in local folder we need git-lfs. This will be installed using command line where jupyter notebook is installed.
the git-lfs can be installed using the following command - 

pip install homebrew #install homebrew

eval "$(/opt/homebrew/bin/brew shellenv)" # initialize brew

brew install git-lfs 

git lfs install # Updated Git hooks. Git LFS initialized.

jupyter lab

In [ ]:
import os


In [ ]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
repo_dir = now.strftime("%d-%m-%Y-%H-%M-%S")
print("date and time =", repo_dir)
if not os.path.exists(repo_dir):
   os.makedirs(repo_dir)

#os.chdir(repo_dir)
  


In [ ]:

from git import Repo  # pip install gitpython
git_url = "https://huggingface.co/bsaurav/results"

Repo.clone_from(git_url, repo_dir)
#git clone git@hf.co:bsaurav/results

In [ ]:
import tarfile 
import shutil

# delete .git, folder and then zip
def remove(path):
    """ param <path> could either be relative or absolute. """
    if os.path.isfile(path) or os.path.islink(path):
        os.remove(path)  # remove the file
    elif os.path.isdir(path):
        shutil.rmtree(path)  # remove dir and all contains
    else:
        raise ValueError("file {} is not a file or dir.".format(path))
remove(repo_dir + "/.git")
remove(repo_dir + "/README.md")
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
        
make_tarfile(repo_dir + ".tar.gz", repo_dir)


# cleanup 
remove(repo_dir)

In [ ]:
#iam_client = boto3.client('iam')
#role = iam_client.get_role(RoleName='role-name-of-your-iam-role-with-right-permissions')['Role']['Arn']
#sess = sagemaker.Session()

In [ ]:
# s3 bucket personalassistantml
import boto3
#s3_client = boto3.client('s3')
#with open(repo_dir + ".tar.gz", "rb") as f:
#    s3_client.upload_fileobj(f, "personalassistantml", repo_dir + ".tar.gz")

session = boto3.Session(profile_name="xxxxxxxx")

sts = session.client("sts")
response = sts.assume_role(
    RoleArn="arn:aws:iam::xxxxxx:role/xxxxx",
    RoleSessionName="xxxxxx"
)
#print(response)
new_session = boto3.Session(aws_access_key_id=response['Credentials']['AccessKeyId'],
                      aws_secret_access_key=response['Credentials']['SecretAccessKey'],
                      aws_session_token=response['Credentials']['SessionToken'])
s3 = new_session.client("s3")
filetoUpload = open(repo_dir + ".tar.gz", 'rb')
respputobject = s3.put_object(
     Body=filetoUpload,
    Bucket='xxxxxx',
    Key=repo_dir + ".tar.gz"
    )
#print(respputobject)
# cleanup
if(respputobject["ResponseMetadata"]["HTTPStatusCode"] == 200) :
   os.remove(repo_dir + ".tar.gz")

# update the build version so that deployment script can pick this up - TBD  

In [ ]:
# the deployment script. later on we need to bifurcate packaging and deployment
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker import get_execution_role

role = get_execution_role()

#iam_client = boto3.client('iam')
#role = iam_client.get_role(RoleName='role-name-of-your-iam-role-with-right-permissions')['Role']['Arn']
#sess = sagemaker.Session()
model_data ="s3://xxxx/" + repo_dir + ".tar.gz"
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=model_data,  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)



